In [86]:
import statsmodels.formula.api as smf
import first
import nsfg
import pandas as pd

In [7]:
live, firsts, others = first.MakeFrames()
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()

In [9]:
inter = results.params['Intercept']
slope = results.params['agepreg']

In [11]:
slope_pvalue = results.pvalues['agepreg']

In [13]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Tue, 26 Jul 2022   Prob (F-statistic):           5.72e-11
Time:                        08:55:52   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
from regression import SummarizeResults
SummarizeResults(results)

Intercept   6.83   (0)
agepreg   0.0175   (5.72e-11)
R^2 0.004738
Std(ys) 1.408
Std(res) 1.405


**std(ys)** is the standard deviation of the dependent variable (the RMSE if you had to guess without any explanatory variables)

**std(res)** is the standard deviation of the residuals (the RMSE if your guesses are informed by explanatory variables)

### Multiple Regression

In [18]:
diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_age = firsts.agepreg.mean() - others.agepreg.mean()

In [20]:
results2 = smf.ols('totalwgt_lb ~ agepreg', data = live).fit()
slope2 = results2.params['agepreg']

In [22]:
slope2

0.017453851471802843

In [24]:
live['isfirst'] = live.birthord == 1 ## New column T/F for first-born
formula = 'totalwgt_lb ~ isfirst'
results3 = smf.ols(formula, data = live).fit()

In [25]:
SummarizeResults(results3)

Intercept   7.33   (0)
isfirst[T.True]   -0.125   (2.55e-05)
R^2 0.00196
Std(ys) 1.408
Std(res) 1.407


In [29]:
formula2 = 'totalwgt_lb ~ isfirst + agepreg'
results4 = smf.ols(formula2, data = live).fit()

In [30]:
SummarizeResults(results4)

Intercept   6.91   (0)
isfirst[T.True]   -0.0698   (0.0253)
agepreg   0.0154   (3.93e-08)
R^2 0.005289
Std(ys) 1.408
Std(res) 1.405


In [32]:
live['agepreg2'] = live.agepreg**2
formula3 = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'
results5 = smf.ols(formula3, data = live).fit()

In [33]:
SummarizeResults(results5)

Intercept   5.69   (1.38e-86)
isfirst[T.True]   -0.0504   (0.109)
agepreg   0.112   (3.23e-07)
agepreg2   -0.00185   (8.8e-06)
R^2 0.007462
Std(ys) 1.408
Std(res) 1.403


### Join

In [37]:
import chap01soln

live = live[live.prglngth>30]
resp = chap01soln.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [38]:
t = []
for name in join.columns:
    try:
        if join[name].var() < 1e-7:
            continue
        
        formula = 'totalwgt_lb ~ agepreg + ' + name
        model = smf.ols(formula, data = join)
        if model.nobs < len(join)/2:
            continue
        
        results = model.fit()
    except (ValueError, TypeError):
        continue
    
    t.append((results.rsquared, name))

In [39]:
t.sort(reverse = True)

for mse, name in t[:30]:
    print(name, mse)

totalwgt_lb 1.0
birthwgt_lb 0.9498127305978009
lbw1 0.3008240784470769
prglngth 0.13012519488625063
wksgest 0.12340041363361054
agecon 0.10203149928156052
mosgest 0.027144274639579802
babysex 0.01855092529394209
race_r 0.016199503586253217
race 0.016199503586253217
nbrnaliv 0.016017752709788113
paydu 0.014003795578114597
rmarout03 0.013430066465713209
birthwgt_oz 0.013102457615706053
anynurse 0.012529022541810764
bfeedwks 0.01219368840449575
totincr 0.011870069031173491
marout03 0.011807801994375033
marcon03 0.011752599354395654
cebow 0.011437770919637158
rmarout01 0.011407737138640295
rmarout6 0.011354138472805753
marout01 0.011269357246806444
hisprace_r 0.011238349302030826
hisprace 0.011238349302030826
mar1diss 0.010961563590751733
fmarcon5 0.0106049646842995
rmarout02 0.010546913206565312
marcon02 0.010481401795534251
fmarout5 0.01046169136737718


In [43]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + ' 'nbrnaliv>1 + paydu==1 + totincr')
results = smf.ols(formula, data=join).fit()

In [46]:
SummarizeResults(results)

Intercept   6.63   (0)
C(race)[T.2]   0.357   (5.43e-29)
C(race)[T.3]   0.266   (2.33e-07)
babysex == 1[T.True]   0.295   (5.39e-29)
nbrnaliv > 1[T.True]   -1.38   (5.1e-37)
paydu == 1[T.True]   0.12   (0.000114)
agepreg   0.00741   (0.0035)
totincr   0.0122   (0.00188)
R^2 0.05999
Std(ys) 1.271
Std(res) 1.232


### Logistic Regression

In [47]:
live, firsts, others = first.MakeFrames()
df = live[live.prglngth > 30]

In [80]:
live['boy'] = (live.babysex == 1).astype(int)

In [83]:
model = smf.logit('boy ~ agepreg', data=live)

In [82]:
join.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth,boy
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833,1
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833,0
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167,1
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167,0
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167,0


In [84]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=live)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692944
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 9034
Model:                          Logit   Df Residuals:                     9028
Method:                           MLE   Df Model:                            5
Date:                Tue, 26 Jul 2022   Pseudo R-squ.:               0.0001664
Time:                        09:49:31   Log-Likelihood:                -6260.1
converged:                       True   LL-Null:                       -6261.1
Covariance Type:            nonrobust   LLR p-value:                    0.8374
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0501      0.102     -0.490      0.624      -0.250       0.150
C(race)[T.2]    -0.0145      0.050     -0.289      0.772      -0.112       0.083
C(race)[T.3]    -0.0184      0.082     -0.225      0.822      -0.179       0.142
agepreg         -0.0027      0.005     -0.503      0.615      -0.013       0.008
hpagelb          0.0052      0.004      1.249      0.212      -0.003       0.013
birthord         0.0058      0.022      0.266      0.790      -0.037       0.048
================================================================================
"""

In [87]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

In [88]:
actual = endog['boy']
baseline = actual.mean()
baseline

0.5066415762674341

In [89]:
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)
sum(true_pos), sum(true_neg)

(3850.0, 755.0)

In [90]:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)
acc

0.5097409785255701